# LLM Queries in DuckDB

This notebook walks through how to call LLMs directly as a UDF in a DuckDB database using [vLLM](https://github.com/vllm-project/vllm) or [OpenAI](https://platform.openai.com/docs/api-reference) as the inference engine.

## Initialize the LLM Engine

Choose between OpenAI or vLLM with a quantized version of Llama-3 8B

In [1]:
# Uncomment the below code to initialize llmsql with OpenAI

# import llmsql
# from llmsql.llm.openai import OpenAI


# llmsql.init(OpenAI(base_url="https://api.openai.com/v1", api_key="<INSERT_OPENAI_KEY>"))


# Uncomment the below code to initialize llmsql with vLLM

import llmsql
from llmsql.llm.vllm import vLLM
from vllm import EngineArgs
args = EngineArgs(model="TechxGenus/Meta-Llama-3-8B-Instruct-GPTQ")

llmsql.init(vLLM(engine_args=args))


Starting vLLM engine...
WARNING 05-01 12:19:36 config.py:169] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-01 12:19:36 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='TechxGenus/Meta-Llama-3-8B-Instruct-GPTQ', speculative_config=None, tokenizer='TechxGenus/Meta-Llama-3-8B-Instruct-GPTQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-01 12:19:36 utils.py:608] Found nccl from library /home/ray/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-01 12:19:37 selector.py:65] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 05-01 12:19:37 selector.py:33] Using XFormers backend.
INFO 05-01 12:19:39 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 05-01 12:19:41 model_runner.py:173] Loading model weights took 5.3472 GB
INFO 05-01 12:19:47 gpu_executor.py:119] # GPU blocks: 2602, # CPU blocks: 2048
INFO 05-01 12:19:49 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-01 12:19:49 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed

## Load the movies dataset as a DuckDB table

In [2]:
# Make sure you import duckdb from llmsql
from llmsql.duckdb import duckdb

# Create a table from the movies dataset
conn = duckdb.connect(database=':memory:', read_only=False)
conn.execute("CREATE TABLE movies AS SELECT * FROM read_csv('./movies_small.csv')")
conn.execute("CREATE TABLE movies_limit as SELECT * FROM movies WHERE review_content IS NOT NULL LIMIT 20")

In [3]:
# View the table

conn.sql("SHOW TABLES")

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ movies       │
│ movies_limit │
└──────────────┘

In [4]:
# Show fields in the movie_limit table

conn.sql("DESCRIBE movies_limit")

┌──────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type │  null   │   key   │ default │  extra  │
│       varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ rotten_tomatoes_link │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ review_content       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ movie_title          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ movie_info           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

## Run sample LLM Queries

### LLMs in Projection Queries

In [7]:
query = (
    "SELECT review_content, LLM('Given a movie review as {review_content}, classify the review as either POSITIVE, NEGATIVE, or NEUTRAL." 
    "Respond with just the category and no other text.', review_content) AS sentiment FROM movies_limit")
conn.sql(query).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│                                              review_content                                              │ sentiment │
│                                                 varchar                                                  │  varchar  │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────┤
│ It's a series of routines within a routine formula, and the result is as tedious as it sounds.           │ NEGATIVE  │
│ A vulgar exercise of terror that, despite its defects, manages to stand out from its delectable predec…  │ POSITIVE  │
│ After the Thin Man hasn't quite the spontaneity and charm of the original, but it's good mystery-comed…  │ POSITIVE  │
│ You never really get angry at it. You just want to shake it up because the elements for a first-class …  │ NEGATIVE  │
│ An excellent film.            

In [9]:
query = (
    "SELECT movie_title, movie_info, review_content, LLM('Given {movie_title}, {movie_info} and a movie review as {review_content}, extract all character names that are mentioned."
    "Respond with just the character names and no other text. If there are no characters, respond with just None', movie_title, movie_info, review_content) AS character_names FROM movies_limit")
conn.sql(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬──────────────────────┬──────────────────────┬─────────────────────────────────────────────────┐
│     movie_title      │      movie_info      │    review_content    │                 character_names                 │
│       varchar        │       varchar        │       varchar        │                     varchar                     │
├──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────────────────────────────────┤
│ Amityville: The Aw…  │ When some footage …  │ It's a series of r…  │ None                                            │
│ Amityville: The Aw…  │ When some footage …  │ A vulgar exercise …  │ None                                            │
│ After the Thin Man   │ Recently returned …  │ After the Thin Man…  │ Nick Charles, Nora, Selma, Robert, David Graham │
│ Silver Streak        │ While on a cross-c…  │ You never really g…  │ Gene Wilder, Jill Clayburgh, Richard Pryor      │
│ Boomerang!           │ Connect

### Filter Queries

We can also use the LLM query as filters, possibly in combination with projections

In [10]:
filter_query = (
    "SELECT movie_title, review_content FROM movies_limit WHERE "
    "LLM('Given a movie review as {review_content}, classify the review as either POSITIVE, NEGATIVE, or NEUTRAL. " 
    "Respond with just the category and no other text.', review_content) == 'POSITIVE'")
conn.sql(filter_query).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│     movie_title      │                                        review_content                                         │
│       varchar        │                                            varchar                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ Amityville: The Aw…  │ A vulgar exercise of terror that, despite its defects, manages to stand out from its delect…  │
│ After the Thin Man   │ After the Thin Man hasn't quite the spontaneity and charm of the original, but it's good my…  │
│ Silver Streak        │ You never really get angry at it. You just want to shake it up because the elements for a f…  │
│ Boomerang!           │ An excellent film.                                                                            │
│ American Gun         │ Martin 

In [12]:
query = (
    "SELECT movie_title, LLM('Given {movie_title} and {movie_info}, determine if this movie is suitable for kids.', movie_title, movie_info) AS kids_suitable "
    "FROM movies_limit WHERE "
    "LLM('Given a movie review as {review_content}, classify the review as either POSITIVE, NEGATIVE, or NEUTRAL." 
    "Respond with just the category and no other text.', review_content) == 'POSITIVE'"
)
conn.sql(query).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────────────────────┬───────────────────────────────────────────────────────────────────┐
│        movie_title        │                           kids_suitable                           │
│          varchar          │                              varchar                              │
├───────────────────────────┼───────────────────────────────────────────────────────────────────┤
│ Amityville: The Awakening │ No                                                                │
│ After the Thin Man        │ No                                                                │
│ Silver Streak             │ Not suitable for kids                                             │
│ Boomerang!                │ No, it's not suitable for kids.                                   │
│ American Gun              │ "No"                                                              │
│ Armstrong                 │ No                                                                │
│ White Fang        

### Aggregate Query

LLM queries can also be used in aggregates.

In [14]:
query = (
    "SELECT movie_title, " 
        "AVG(CAST(LLM("
            "'Given a movie review as {review_content}, score the movie either as 1, 2, 3, with 3 as the highest. Return just the score and nothing else.', movie_title, movie_info) "
        "AS integer)) AS average_review_score "
    "FROM movies_limit GROUP BY movie_title "
)
conn.sql(query).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────────────────┬──────────────────────┐
│        movie_title         │ average_review_score │
│          varchar           │        double        │
├────────────────────────────┼──────────────────────┤
│ Amityville: The Awakening  │   2.6666666666666665 │
│ Silent Night, Deadly Night │                  2.0 │
│ Hoodlum                    │                  2.0 │
│ Boomerang!                 │                  2.0 │
│ Alexandra's Project        │                  2.0 │
│ Meet Bill                  │                  2.0 │
│ After the Thin Man         │                  1.0 │
│ Death in Love              │                  2.0 │
│ American Gun               │                  1.0 │
│ Armstrong                  │                  2.0 │
│ Badland                    │                  2.0 │
│ White Fang                 │                  1.0 │
│ Stolen                     │                  2.0 │
│ The Assignment             │                  2.0 │
│ The Jazz Singer           